# Severity prediction of bug reports

## Baseline

- Preprocess: 
- Embedding using:
- Algorithms for binary classification:
- Metrics: 

## Code

### Boot

In [1]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

In [49]:
import pandas as pd
import numpy as np

import string
import re

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.naive_bayes import BernoulliNB, GaussianNB, ComplementNB, MultinomialNB
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [50]:
from baseline_functions import filter_bug_severity, create_binary_feature, remove_urls_and_codes

### Data

In [51]:
bug_reports_ = pd.read_json('data/eclipse_clear.json', lines=True)
bug_reports_.info()
############ fazer um script para pegar o summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361006 entries, 0 to 361005
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   _id           361006 non-null  object
 1   bug_id        361006 non-null  int64 
 2   product       361006 non-null  object
 3   description   361006 non-null  object
 4   bug_severity  361006 non-null  object
 5   dup_id        361006 non-null  object
 6   short_desc    361006 non-null  object
 7   priority      361006 non-null  object
 8   version       361006 non-null  object
 9   component     361006 non-null  object
 10  delta_ts      361006 non-null  object
 11  bug_status    361006 non-null  object
 12  creation_ts   361006 non-null  object
 13  resolution    361006 non-null  object
dtypes: int64(1), object(13)
memory usage: 38.6+ MB


In [52]:
bug_reports = bug_reports_[:60000]
bug_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           60000 non-null  object
 1   bug_id        60000 non-null  int64 
 2   product       60000 non-null  object
 3   description   60000 non-null  object
 4   bug_severity  60000 non-null  object
 5   dup_id        60000 non-null  object
 6   short_desc    60000 non-null  object
 7   priority      60000 non-null  object
 8   version       60000 non-null  object
 9   component     60000 non-null  object
 10  delta_ts      60000 non-null  object
 11  bug_status    60000 non-null  object
 12  creation_ts   60000 non-null  object
 13  resolution    60000 non-null  object
dtypes: int64(1), object(13)
memory usage: 6.4+ MB


In [39]:
bug_reports.bug_severity.unique()

array(['normal', 'major', 'enhancement', 'critical', 'minor', 'trivial',
       'blocker'], dtype=object)

### Pipeline

In [44]:
filter_df = filter_bug_severity(bug_reports, col='bug_severity')
filter_df.head(3)

,_id,bug_id,description,bug_severity
44,{'$oid': '52e9b44954dc1c25ebdb1f11'},43,I have a project (Junk) that has been released...,major
162,{'$oid': '52e9b44f54dc1c25ebdb1f87'},163,AK (6/12/01 4:55:24 PM)\n\ti got this exceptio...,critical
192,{'$oid': '52e9b45054dc1c25ebdb1fa5'},194,"1) Add a global ignore pattern ""BazProject""\n2...",major


In [45]:
binary_df = create_binary_feature(filter_df, col='bug_severity')
binary_df.head(3)

,_id,bug_id,description,bug_severity,binary_severity
44,{'$oid': '52e9b44954dc1c25ebdb1f11'},43,I have a project (Junk) that has been released...,major,1
162,{'$oid': '52e9b44f54dc1c25ebdb1f87'},163,AK (6/12/01 4:55:24 PM)\n\ti got this exceptio...,critical,1
192,{'$oid': '52e9b45054dc1c25ebdb1fa5'},194,"1) Add a global ignore pattern ""BazProject""\n2...",major,1


In [36]:
###############
# check as remover without flag, and as it is in the data
def remove_code_snippets(text):
    # Remove programming code snippets enclosed in triple backticks
    code_pattern = r'```(?:[^`]+|`(?!``))*```'
    text_without_code = re.sub(code_pattern, 'CODE', text)

    # Remove programming code snippets enclosed in single backticks
    text_without_code = re.sub(r'`[^`]+`', '', text_without_code)

    return text_without_code

def remove_urls_and_codes(dataframe, col='description'):
    test = dataframe[col].apply(lambda text: print(type(text)))
    dataframe[col] = dataframe[col].apply(lambda text: re.sub(r'http\S+', '', text))
    dataframe[col] = dataframe[col].apply(lambda text: remove_code_snippets(text))
    return dataframe

In [ ]:
url_df = remove_urls_and_codes(binary_df, col='description')
url_df.head(3)

In [15]:
# adicionar o tratamento do campo vazio ??
remove_urls_and_codes(bug_reports, col='description')

In [7]:
def preprocess_text(dataframe, col='description'):
    bug_reports_copy = dataframe.copy()
    
    tokens = bug_reports_copy[col].apply(word_tokenize)
    
    # Get the set of stopwords
    stop_words = set(stopwords.words('english'))

    # Define the special characters to remove
    # in the article doesn't include this step
    special_characters = set(string.punctuation)
    special_characters.add('``')
    special_characters.add("''")
    # there is also:  "n't" / checkar
    
    # Initialize the stemmer
    stemmer = PorterStemmer()
    
    # Apply stop-word removal and stemming
    filtered_texts = []
    for tokens in tokens:
        filtered_tokens = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words and token not in special_characters]
        # filtered_texts.append(filtered_tokens)
        filtered_texts.append(' '.join(filtered_tokens))
    #print('\nfiltered_texts', filtered_texts)
    bug_reports_copy['preprocess_desc'] = filtered_texts
    
    return bug_reports_copy

In [ ]:
# checar se estou tratando os enters
# qual a melhor ordem de colocar o tratamento dos enters
# testar o passo de remover URL e CODE
# checar a estrutura da primeira referencia
# fazer os testes em pedaços dos bugs e ir salvando os arquivos (chuncks)

In [46]:
test = preprocess_text(bug_reports, col='description')
test.to_csv('my_dataframe.csv', index=False)

In [8]:
pipeline = Pipeline([
    ('bug_severity_filter', FunctionTransformer(filter_bug_severity, kw_args={'col': 'bug_severity'})),
    ('binary_feature', FunctionTransformer(create_binary_feature, kw_args={'col': 'bug_severity'})),
    ('remove_url_and_code', FunctionTransformer(remove_urls_and_codes, kw_args={'col': 'description'})),
    ('preprocessor', FunctionTransformer(preprocess_text, kw_args={'col': 'description'})),
    # ('embedding', embedding),
    # ('classifier', classifier)
])

In [ ]:
pipeline.fit(bug_reports)

# Get the transformed data at the filter_bug_severity step
filter_bug_severity_result = pipeline.named_steps['bug_severity_filter'].transform(bug_reports) # ver sobre a seleção das colunas
print('First step')
print('Columns:', filter_bug_severity_result.columns)
print('Return type:', type(filter_bug_severity_result))
print("Severities's kind:", filter_bug_severity_result.bug_severity.unique(), '\n')

# Get the transformed data at the binary_feature step
binary_feature_result = pipeline.named_steps['binary_feature'].transform(filter_bug_severity_result)
print('Second step')
print('Columns:', binary_feature_result.columns)
print('Return type:', type(binary_feature_result))
print("Severities's kind:", binary_feature_result.binary_severity.unique(), '\n')

# Get the transformed data at the preprocessor
preprocessor_result = pipeline.named_steps['preprocessor'].transform(binary_feature_result)
#print('Third step')
#print('Columns:', preprocessor_result.columns)
#print('Return type:', type(preprocessor_result))
#print("Severities's kind:", preprocessor_result.head(3).preprocess_desc, '\n')

#### Pipeline Naive Bayes

Article of A. Lamkanfi (Comparing...)

In [153]:
# Define the Binary representation (CountVectorizer)
binary_representation = CountVectorizer()

# Define the Naive Bayes classifier
naive_bayes_classifier = BernoulliNB()

In [9]:
data_preprocess_pipeline = Pipeline([
    ('bug_severity_filter', FunctionTransformer(filter_bug_severity, kw_args={'col': 'bug_severity'})),
    ('binary_feature', FunctionTransformer(create_binary_feature, kw_args={'col': 'bug_severity'})),
    ('preprocessor', FunctionTransformer(preprocess_text, kw_args={'col': 'description'})),
])

In [10]:
data_preprocessed = data_preprocess_pipeline.transform(bug_reports)

In [11]:
data_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11232 entries, 44 to 59999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              11232 non-null  object
 1   bug_id           11232 non-null  int64 
 2   description      11232 non-null  object
 3   bug_severity     11232 non-null  object
 4   binary_severity  11232 non-null  int64 
 5   preprocess_desc  11232 non-null  object
dtypes: int64(2), object(4)
memory usage: 614.2+ KB


In [187]:
# Create bag-of-words representation
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(data_preprocessed['preprocess_desc'])
# print('bag_of_words', bag_of_words)

# Get the feature names
feature_names = vectorizer.get_feature_names_out()
print(len(feature_names), feature_names)

# Print the bag-of-words representation
print(len(bag_of_words.toarray()), '\n', bag_of_words.toarray())

63651 ['00' '000' '0000' ... 'あいうえお' '九月' '論理ﾋﾞｭｰ']
11232 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [188]:
X = bag_of_words.toarray()
y = data_preprocessed['binary_severity'].values

In [189]:
#k-fold cross-validation
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [160]:
############# Bernouli

# Initialize lists to store train and test accuracies
train_accuracies = []
test_accuracies = []

# Iterate through each fold
for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    naive_bayes_classifier.fit(X_train, y_train)
    
    y_pred = naive_bayes_classifier.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)

    y_train_pred = naive_bayes_classifier.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    # Print accuracies for this fold
    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Calculate and print mean accuracies across all folds
mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}')

Fold 1 - Train Accuracy: 0.5598, Test Accuracy: 0.5142
Fold 2 - Train Accuracy: 0.5540, Test Accuracy: 0.5311
Fold 3 - Train Accuracy: 0.5579, Test Accuracy: 0.5236
Fold 4 - Train Accuracy: 0.5560, Test Accuracy: 0.5076
Fold 5 - Train Accuracy: 0.5568, Test Accuracy: 0.5467
Fold 6 - Train Accuracy: 0.5612, Test Accuracy: 0.5200
Fold 7 - Train Accuracy: 0.5560, Test Accuracy: 0.4871
Fold 8 - Train Accuracy: 0.5572, Test Accuracy: 0.5503
Fold 9 - Train Accuracy: 0.5521, Test Accuracy: 0.5165
Fold 10 - Train Accuracy: 0.5611, Test Accuracy: 0.5316
Mean Train Accuracy: 0.5572, Mean Test Accuracy: 0.5229


In [190]:
############# GaussianNB

naive_bayes_classifier = GaussianNB()

# Initialize lists to store train and test accuracies
train_accuracies = []
test_accuracies = []

# Iterate through each fold
for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    naive_bayes_classifier.fit(X_train, y_train)
    
    y_pred = naive_bayes_classifier.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)

    y_train_pred = naive_bayes_classifier.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    # Print accuracies for this fold
    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Calculate and print mean accuracies across all folds
mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}')

Fold 1 - Train Accuracy: 0.8635, Test Accuracy: 0.6637
Fold 2 - Train Accuracy: 0.8624, Test Accuracy: 0.6619
Fold 3 - Train Accuracy: 0.8614, Test Accuracy: 0.6759
Fold 4 - Train Accuracy: 0.8639, Test Accuracy: 0.6429
Fold 5 - Train Accuracy: 0.8633, Test Accuracy: 0.6687
Fold 6 - Train Accuracy: 0.8616, Test Accuracy: 0.6723
Fold 7 - Train Accuracy: 0.8645, Test Accuracy: 0.6447
Fold 8 - Train Accuracy: 0.8624, Test Accuracy: 0.6741
Fold 9 - Train Accuracy: 0.8615, Test Accuracy: 0.6723
Fold 10 - Train Accuracy: 0.8641, Test Accuracy: 0.6759
Mean Train Accuracy: 0.8629, Mean Test Accuracy: 0.6652


In [191]:
############# GaussianNB

naive_bayes_classifier = GaussianNB()

# Initialize lists to store train and test accuracies
train_accuracies = []
test_accuracies = []

# Iterate through each fold
for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    naive_bayes_classifier.fit(X_train, y_train)
    
    y_pred = naive_bayes_classifier.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)

    y_train_pred = naive_bayes_classifier.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)
    
    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(f'Fold {fold + 1} - ROC AUC: {roc_auc:.4f}')

    # Print accuracies for this fold
    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Calculate and print mean accuracies across all folds
mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}')

Fold 1 - ROC AUC: 0.6267
Fold 1 - Train Accuracy: 0.8635, Test Accuracy: 0.6637
Fold 2 - ROC AUC: 0.6445
Fold 2 - Train Accuracy: 0.8624, Test Accuracy: 0.6619
Fold 3 - ROC AUC: 0.6506
Fold 3 - Train Accuracy: 0.8614, Test Accuracy: 0.6759
Fold 4 - ROC AUC: 0.6226
Fold 4 - Train Accuracy: 0.8639, Test Accuracy: 0.6429
Fold 5 - ROC AUC: 0.6345
Fold 5 - Train Accuracy: 0.8633, Test Accuracy: 0.6687
Fold 6 - ROC AUC: 0.6375
Fold 6 - Train Accuracy: 0.8616, Test Accuracy: 0.6723
Fold 7 - ROC AUC: 0.6117
Fold 7 - Train Accuracy: 0.8645, Test Accuracy: 0.6447
Fold 8 - ROC AUC: 0.6450
Fold 8 - Train Accuracy: 0.8624, Test Accuracy: 0.6741
Fold 9 - ROC AUC: 0.6311
Fold 9 - Train Accuracy: 0.8615, Test Accuracy: 0.6723
Fold 10 - ROC AUC: 0.6563
Fold 10 - Train Accuracy: 0.8641, Test Accuracy: 0.6759
Mean Train Accuracy: 0.8629, Mean Test Accuracy: 0.6652


Mean AUC: 0.63605

In [165]:
############# ComplementNB

naive_bayes_classifier = ComplementNB()

# Initialize lists to store train and test accuracies
train_accuracies = []
test_accuracies = []

# Iterate through each fold
for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    naive_bayes_classifier.fit(X_train, y_train)
    
    y_pred = naive_bayes_classifier.predict(X_test)

    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)

    y_train_pred = naive_bayes_classifier.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    # Print accuracies for this fold
    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Calculate and print mean accuracies across all folds
mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}')

Fold 1 - Train Accuracy: 0.4556, Test Accuracy: 0.4297
Fold 2 - Train Accuracy: 0.4502, Test Accuracy: 0.4422
Fold 3 - Train Accuracy: 0.4503, Test Accuracy: 0.4461
Fold 4 - Train Accuracy: 0.4500, Test Accuracy: 0.4354
Fold 5 - Train Accuracy: 0.4532, Test Accuracy: 0.4337
Fold 6 - Train Accuracy: 0.4552, Test Accuracy: 0.4408
Fold 7 - Train Accuracy: 0.4528, Test Accuracy: 0.4159
Fold 8 - Train Accuracy: 0.4502, Test Accuracy: 0.4550
Fold 9 - Train Accuracy: 0.4540, Test Accuracy: 0.4203
Fold 10 - Train Accuracy: 0.4528, Test Accuracy: 0.4497
Mean Train Accuracy: 0.4524, Mean Test Accuracy: 0.4369


In [163]:
def bag_of_words(texts):
    vectorizer = CountVectorizer()
    bag_of_words = vectorizer.fit_transform(data_preprocessed['preprocess_desc'])

    return bag_of_words.toarray()

In [167]:
complete_pipeline = Pipeline([
    ('embedding', FunctionTransformer(bag_of_words)),
    ('classifier', naive_bayes_classifier)
])

In [132]:
# k-fold cross-validation
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Calculate accuracy using cross_val_score
accuracy_scores = cross_val_score(complete_pipeline, X, y, cv=skf, scoring='accuracy')

# Calculate and print mean accuracy across all folds
# mean_accuracy = accuracy_scores.mean()
# print(f'Mean Accuracy: {mean_accuracy:.4f}')

2861
2861
2861
2861
2861
2861
2861
2861
2861
2862


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\sklearn\pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\dessa\anaconda3\envs\Master\lib\site-packages\sklearn\pipeline.py", line 370, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "

In [ ]:
# Apply initial steps on the whole dataset
data_preprocessed = pipeline.named_steps['preprocessor'].transform(data)

### Pipeline KNN

Article of A. Lamkanfi (Comparing...)

In [ ]:
# data_preprocessed = data_preprocess_pipeline.transform(bug_reports)

In [12]:
# Embedding
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=5)

In [13]:
data_preprocessed.columns

Index(['_id', 'bug_id', 'description', 'bug_severity', 'binary_severity',
       'preprocess_desc'],
      dtype='object')

In [14]:
#k-fold cross-validation
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

X = data_preprocessed['preprocess_desc']
y = data_preprocessed['binary_severity'].values

In [15]:
train_accuracies = []
test_accuracies = []
aucs = []

for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    # Fit and transform TF-IDF vectors
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    knn_classifier.fit(X_train_tfidf, y_train)
    y_pred = knn_classifier.predict(X_test_tfidf)

    # Calculate and store test accuracy
    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)
    
    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    print(f'Fold {fold + 1} - ROC AUC: {roc_auc:.4f}')

    # Predict on the training data and calculate train accuracy
    y_train_pred = knn_classifier.predict(X_train_tfidf)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
mean_aucs = sum(aucs) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}, Mean AUC: {mean_aucs:.4f}')

Fold 1 - ROC AUC: 0.5014
Fold 1 - Train Accuracy: 0.7655, Test Accuracy: 0.7580
Fold 2 - ROC AUC: 0.4995
Fold 2 - Train Accuracy: 0.7649, Test Accuracy: 0.7571
Fold 3 - ROC AUC: 0.5056
Fold 3 - Train Accuracy: 0.7641, Test Accuracy: 0.7631
Fold 4 - ROC AUC: 0.5031
Fold 4 - Train Accuracy: 0.7642, Test Accuracy: 0.7614
Fold 5 - ROC AUC: 0.4988
Fold 5 - Train Accuracy: 0.7648, Test Accuracy: 0.7587
Fold 6 - ROC AUC: 0.5001
Fold 6 - Train Accuracy: 0.7654, Test Accuracy: 0.7578
Fold 7 - ROC AUC: 0.5031
Fold 7 - Train Accuracy: 0.7645, Test Accuracy: 0.7605
Fold 8 - ROC AUC: 0.5094
Fold 8 - Train Accuracy: 0.7643, Test Accuracy: 0.7622
Fold 9 - ROC AUC: 0.5050
Fold 9 - Train Accuracy: 0.7643, Test Accuracy: 0.7614
Fold 10 - ROC AUC: 0.5088
Fold 10 - Train Accuracy: 0.7644, Test Accuracy: 0.7614
Mean Train Accuracy: 0.7646, Mean Test Accuracy: 0.7602, Mean AUC: 0.5035


### Pipeline SVM

Article of A. Lamkanfi (Comparing...)

In [20]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create an SVM classifier
svm_classifier = SVC(C=100.0, kernel='rbf', gamma=0.001, probability=True) # how set cost=100.0?

# k-fold cross-validation
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

X = data_preprocessed['preprocess_desc']
y = data_preprocessed['binary_severity'].values

In [21]:
train_accuracies = []
test_accuracies = []
aucs = []

for fold, (train_indices, test_indices) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    # Fit and transform TF-IDF vectors
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    svm_classifier.fit(X_train_tfidf, y_train)
    y_pred = svm_classifier.predict(X_test_tfidf)

    # Calculate and store test accuracy
    test_accuracy = accuracy_score(y_test, y_pred)
    test_accuracies.append(test_accuracy)
    
    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test, svm_classifier.predict_proba(X_test_tfidf)[:, 1])
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    print(f'Fold {fold + 1} - ROC AUC: {roc_auc:.4f}')

    # Predict on the training data and calculate train accuracy
    y_train_pred = svm_classifier.predict(X_train_tfidf)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracies.append(train_accuracy)

    print(f'Fold {fold + 1} - Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')

mean_train_accuracy = sum(train_accuracies) / k
mean_test_accuracy = sum(test_accuracies) / k
mean_aucs = sum(aucs) / k
print(f'Mean Train Accuracy: {mean_train_accuracy:.4f}, Mean Test Accuracy: {mean_test_accuracy:.4f}, Mean AUC: {mean_aucs:.4f}')

Fold 1 - ROC AUC: 0.7914
Fold 1 - Train Accuracy: 0.7705, Test Accuracy: 0.7714
Fold 2 - ROC AUC: 0.7779
Fold 2 - Train Accuracy: 0.7727, Test Accuracy: 0.7625
Fold 3 - ROC AUC: 0.7896
Fold 3 - Train Accuracy: 0.7706, Test Accuracy: 0.7649
Fold 4 - ROC AUC: 0.8102
Fold 4 - Train Accuracy: 0.7705, Test Accuracy: 0.7703
Fold 5 - ROC AUC: 0.8145
Fold 5 - Train Accuracy: 0.7703, Test Accuracy: 0.7658
Fold 6 - ROC AUC: 0.8014
Fold 6 - Train Accuracy: 0.7715, Test Accuracy: 0.7685
Fold 7 - ROC AUC: 0.7894
Fold 7 - Train Accuracy: 0.7709, Test Accuracy: 0.7658
Fold 8 - ROC AUC: 0.7957
Fold 8 - Train Accuracy: 0.7728, Test Accuracy: 0.7631
Fold 9 - ROC AUC: 0.7996
Fold 9 - Train Accuracy: 0.7727, Test Accuracy: 0.7614
Fold 10 - ROC AUC: 0.7802
Fold 10 - Train Accuracy: 0.7703, Test Accuracy: 0.7640
Mean Train Accuracy: 0.7713, Mean Test Accuracy: 0.7658, Mean AUC: 0.7950


# Production

In [ ]:
# Create the pipeline for production
production_pipeline = Pipeline([
    ('bug_severity_filter', FunctionTransformer(filter_bug_severity, kw_args={'col': 'bug_severity'})),
    ('binary_feature', FunctionTransformer(create_binary_feature, kw_args={'col': 'bug_severity'})),
    ('preprocessing', TextPreprocessor()),
    ('embedding', Word2VecEmbedding()),
    ('classifier', SVC())
])

# Fit the production pipeline on the entire dataset
production_pipeline.fit(X_text, y)  # Assuming X_text and y are already defined

# Make predictions on new, unseen data
new_data = ['New sentence 1.', 'New sentence 2.', ...]
predictions = production_pipeline.predict(new_data)
print(predictions)
# In this production pipeline, I've removed the data splitting step and fitted the pipeline on the entire dataset.
# Then, I can use this pipeline to make predictions on new, unseen data by calling the predict method with the new data. This is a common approach in production when you're deploying a trained model for making predictions on real-world data.





